# Star Wars Analytics with DuckDB

Explore the Star Wars universe using DuckDB for fast analytical queries, joins, and data export to Parquet format.

## Connect to DuckDB

Create an in-memory database for quick analysis.

In [1]:
import duckdb

con = duckdb.connect(database=":memory:")
print("Connected to DuckDB in-memory database")

Connected to DuckDB in-memory database


## Create Star Wars Tables

Set up characters and planets tables with sample data.

In [2]:
con.execute("""
CREATE TABLE characters(
  name VARCHAR,
  affiliation VARCHAR,
  species VARCHAR,
  homeworld VARCHAR,
  midichlorians INTEGER
);
""")

con.execute("""
INSERT INTO characters VALUES
  ('Luke Skywalker','Jedi','Human','Tatooine',15000),
  ('Leia Organa','Rebel','Human','Alderaan',4000),
  ('Han Solo','Rebel','Human','Corellia',3000),
  ('Darth Vader','Sith','Human','Tatooine',20000),
  ('Yoda','Jedi','Unknown','Dagobah',17500),
  ('Obi-Wan Kenobi','Jedi','Human','Stewjon',16000),
  ('Emperor Palpatine','Sith','Human','Naboo',19000);
""")

con.execute("""
CREATE TABLE planets(
  name VARCHAR,
  climate VARCHAR,
  region VARCHAR
);
""")

con.execute("""
INSERT INTO planets VALUES
  ('Tatooine','desert','Outer Rim'),
  ('Alderaan','temperate','Core'),
  ('Corellia','temperate','Core'),
  ('Dagobah','swamp','Outer Rim'),
  ('Stewjon','temperate','Unknown'),
  ('Naboo','temperate','Mid Rim'),
  ('Coruscant','urban','Core');
""")

print("Tables created and seeded with Star Wars data")

Tables created and seeded with Star Wars data


## Basic Queries and Filtering

Explore the data with SELECT statements and filters.

In [3]:
print("All characters:")
print(con.execute("SELECT * FROM characters LIMIT 5").fetchall())

print("\nJedi characters (sorted by midichlorians):")
print(con.execute("""
  SELECT name, homeworld, midichlorians
  FROM characters
  WHERE affiliation = 'Jedi'
  ORDER BY midichlorians DESC
""").fetchall())

print("\nCharacters with 'sky' in their name:")
print(con.execute("""
  SELECT name, affiliation
  FROM characters
  WHERE name ILIKE '%sky%'
""").fetchall())

All characters:
[('Luke Skywalker', 'Jedi', 'Human', 'Tatooine', 15000), ('Leia Organa', 'Rebel', 'Human', 'Alderaan', 4000), ('Han Solo', 'Rebel', 'Human', 'Corellia', 3000), ('Darth Vader', 'Sith', 'Human', 'Tatooine', 20000), ('Yoda', 'Jedi', 'Unknown', 'Dagobah', 17500)]

Jedi characters (sorted by midichlorians):
[('Yoda', 'Dagobah', 17500), ('Obi-Wan Kenobi', 'Stewjon', 16000), ('Luke Skywalker', 'Tatooine', 15000)]

Characters with 'sky' in their name:
[('Luke Skywalker', 'Jedi')]


## Aggregations and Grouping

Analyze data patterns using GROUP BY operations.

In [4]:
print("Count by affiliation:")
print(con.execute("""
  SELECT affiliation, COUNT(*) AS character_count
  FROM characters
  GROUP BY affiliation
  ORDER BY character_count DESC
""").fetchall())

print("\nAverage midichlorians by affiliation:")
print(con.execute("""
  SELECT affiliation, AVG(midichlorians)::INTEGER AS avg_midichlorians
  FROM characters
  GROUP BY affiliation
  ORDER BY avg_midichlorians DESC
""").fetchall())

Count by affiliation:
[('Jedi', 3), ('Rebel', 2), ('Sith', 2)]

Average midichlorians by affiliation:
[('Sith', 19500), ('Jedi', 16167), ('Rebel', 3500)]


## JOIN Operations

Combine characters and planets data for enriched analysis.

In [5]:
print("Characters with planet information:")
print(con.execute("""
  SELECT c.name, c.affiliation, c.homeworld, p.climate, p.region
  FROM characters c
  LEFT JOIN planets p ON c.homeworld = p.name
  ORDER BY c.name
""").fetchall())

print("\nCharacter count by planet climate:")
print(con.execute("""
  SELECT p.climate, COUNT(*) AS character_count
  FROM characters c
  JOIN planets p ON c.homeworld = p.name
  GROUP BY p.climate
  ORDER BY character_count DESC
""").fetchall())

Characters with planet information:
[('Darth Vader', 'Sith', 'Tatooine', 'desert', 'Outer Rim'), ('Emperor Palpatine', 'Sith', 'Naboo', 'temperate', 'Mid Rim'), ('Han Solo', 'Rebel', 'Corellia', 'temperate', 'Core'), ('Leia Organa', 'Rebel', 'Alderaan', 'temperate', 'Core'), ('Luke Skywalker', 'Jedi', 'Tatooine', 'desert', 'Outer Rim'), ('Obi-Wan Kenobi', 'Jedi', 'Stewjon', 'temperate', 'Unknown'), ('Yoda', 'Jedi', 'Dagobah', 'swamp', 'Outer Rim')]

Character count by planet climate:
[('temperate', 4), ('desert', 2), ('swamp', 1)]


## Export to Parquet and Query Files

Save query results to Parquet format and query them directly.

In [6]:
con.execute("""
  COPY (
    SELECT name, homeworld, midichlorians
    FROM characters
    WHERE affiliation = 'Jedi'
  ) TO 'jedi.parquet' (FORMAT 'parquet');
""")

print("Jedi data exported to jedi.parquet")

print("\nQuerying Parquet file directly:")
print(con.execute("SELECT * FROM 'jedi.parquet' ORDER BY midichlorians DESC").fetchall())

con.execute("""
  COPY (
    SELECT c.name, c.affiliation, p.climate, p.region
    FROM characters c
    JOIN planets p ON c.homeworld = p.name
  ) TO 'characters_with_planets.parquet' (FORMAT 'parquet');
""")

print("\nCharacters with planets exported to characters_with_planets.parquet")
print("File contents:")
print(con.execute("SELECT * FROM 'characters_with_planets.parquet'").fetchall())

Jedi data exported to jedi.parquet

Querying Parquet file directly:
[('Yoda', 'Dagobah', 17500), ('Obi-Wan Kenobi', 'Stewjon', 16000), ('Luke Skywalker', 'Tatooine', 15000)]

Characters with planets exported to characters_with_planets.parquet
File contents:
[('Luke Skywalker', 'Jedi', 'desert', 'Outer Rim'), ('Leia Organa', 'Rebel', 'temperate', 'Core'), ('Han Solo', 'Rebel', 'temperate', 'Core'), ('Darth Vader', 'Sith', 'desert', 'Outer Rim'), ('Yoda', 'Jedi', 'swamp', 'Outer Rim'), ('Obi-Wan Kenobi', 'Jedi', 'temperate', 'Unknown'), ('Emperor Palpatine', 'Sith', 'temperate', 'Mid Rim')]
